In [1]:
import json
import math
import pickle
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from cachetools import Cache

In [2]:
# Load and preprocess the data
def load_data(json_files):
    articles = []
    for file in json_files:
        with open(file, 'r') as f:
            articles.extend(json.load(f))
    return articles

In [16]:
# Build the inverted index and calculate TF-IDF scores
def build_inverted_index_and_tfidf_scores(articles, cache):
    inverted_index = defaultdict(list)
    tfidf_scores = []
    total_articles = len(articles)
    stemmer = PorterStemmer()
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    
    for i, article in enumerate(articles):
        text = article['heading'] + ' ' + article['description']
        tokens = preprocess_text(text, stemmer, stop_words)
        tfidf = cache.get(i)
        
        if tfidf is None:
            tfidf = calculate_tfidf(tokens, inverted_index, total_articles)
            cache[i] = tfidf
        
        for term in tokens:
            inverted_index[term].append(i)
        
        tfidf_scores.append((i, tfidf))
    
    return inverted_index, tfidf_scores

In [24]:

# Preprocess the text
def preprocess_text(text, stemmer, stop_words):
    punkt=nltk.download('punkt')
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem(token) for token in tokens if ( token.isalnum() and token not in stop_words)]
    return tokens


In [28]:
# Calculate TF-IDF scores
def calculate_tfidf(tokens, inverted_index, total_articles):
    tfidf = defaultdict(float)
    
    for term in set(tokens):
        tf = tokens.count(term) / len(tokens)
        df = len(inverted_index[term])
        idf = math.log((total_articles + 1) / (df + 1)) + 1
        tfidf[term] = tf * idf
    
    return tfidf


In [6]:

# Process user query
def process_query(query, stemmer, stop_words):
    tokens = preprocess_text(query, stemmer, stop_words)
    return tokens


In [7]:
# Retrieve relevant documents
def retrieve_documents(query_tokens, inverted_index, tfidf_scores):
    relevant_docs = defaultdict(float)
    
    for term in query_tokens:
        doc_ids = inverted_index[term]
        for doc_id in doc_ids:
            relevant_docs[doc_id] += tfidf_scores[doc_id][1][term]
    
    return relevant_docs

In [30]:

# Rank and display results
def rank_and_display_results(relevant_docs, articles):
    ranked_docs = sorted(relevant_docs.items(), key=lambda x: x[1],reverse=True)
    
    for doc_id, relevance in ranked_docs:
        article = articles[doc_id]
        print(f"Heading: {article['heading']}")
        print(f"URL: {article['url']}")
        print(f"Relevance Score: {relevance}")
        print()

In [26]:
# Main function
def main():
    print("starting program...")
    # List of JSON files containing news articles
    json_files = ['news1.json', 'news2.json']  # Update with your file names

    # Load and preprocess the data
    articles = load_data(json_files)

    # Initialize the cache
    cache = Cache(maxsize=10000)  # Set an appropriate cache size

    # Build the inverted index and calculate TF-IDF scores
    inverted_index, tfidf_scores = build_inverted_index_and_tfidf_scores(articles, cache)

    while True:
        # Process user query
        query = input("Enter your query: ")
        if(query=="q"):
            break
        stemmer = PorterStemmer()
        
        stop_words = set(stopwords.words('english'))
        query_tokens = process_query(query, stemmer, stop_words)

        # Retrieve relevant documents
        relevant_docs = retrieve_documents(query_tokens, inverted_index, tfidf_scores)

        # Rank and display results
        rank_and_display_results(relevant_docs, articles)

In [ ]:
main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming

starting program...
Enter your query: Indian courses make it to Top 100 QS World University Rankings


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Heading: 44 Indian courses make it to Top 100 QS World University Rankings
URL: https://timesofindia.indiatimes.com/topic/top-100-in-the-world/newshttps://timesofindia.indiatimes.com/india/44-indian-courses-make-it-to-top-100-qs-world-university-rankings/articleshow/98924111.cms
Relevance Score: 5.227030088359069

Heading: Indian Institute of Technology Guwahati among world's top univs in 14 subjects: Report
URL: https://timesofindia.indiatimes.com/topic/top-100-in-the-world/newshttps://timesofindia.indiatimes.com/city/guwahati/indian-institute-of-technology-guwahati-among-worlds-top-univs-in-14-subjects-report/articleshow/99050401.cms
Relevance Score: 2.329506650981936

Heading: QS univ rankings: 44 desi courses in world’s top 100, up from 35 last year
URL: https://timesofindia.indiatimes.com/topic/top-100-in-the-world/newshttps://timesofindia.indiatimes.com/city/mumbai/qs-univ-rankings-44-desi-courses-in-worlds-top-100-up-from-35-last-year/articleshow/98928479.cms
Relevance Score: 1.